# AlexNet

## Models

![](alexnet.png)

- used normalization and ReLU rather than sigmoid, hence it is much easier to train
- heavy data augumentation


In [1]:
import tensorflow as tf
from run_model import run_model_fn, per_device_batch_size
from utils.cifar10_utils import input_fn
from models_utils import conv2d_fixed_padding

In [2]:
DATA_DIR = "/home/karen/workspace/data/cifar10_data"
BATCH_SIZE = 128
NUM_EPOCHS = 1000

In [18]:
def alexnet_model(inputs, is_training, use_batchnorm, data_format, name="alexnet"):
    # Inputs should be images with dimension (H, W)
    inputs = conv2d_fixed_padding(inputs, 96, 11, 4, data_format)
    inputs = tf.layers.max_pooling2d(inputs,pool_size=3, strides=2)
    inputs = tf.layers.batch_normalization(inputs, training=is_training)
    
    paddings = tf.constant([[1, 1],[2, 2,], [2, 2], [1, 1]])
    inputs = tf.pad(inputs, paddings, "CONSTANT")
    inputs = conv2d_fixed_padding(inputs, 256, 5, 1, data_format)
    inputs = tf.layers.max_pooling2d(inputs,pool_size=3, strides=2)
    inputs = tf.layers.batch_normalization(inputs, training=is_training)
    
    inputs = conv2d_fixed_padding(inputs, 384, 3, 1, data_format)
    inputs = conv2d_fixed_padding(inputs, 384, 3, 1, data_format)
    inputs = conv2d_fixed_padding(inputs, 256, 3, 1, data_format)

    inputs = tf.layers.max_pooling2d(inputs,pool_size=3, strides=2)

    inputs = tf.layers.flatten(inputs)
    
    inputs = tf.layers.dense(inputs, 4096)
    inputs = tf.layers.dense(inputs, 4096)
    
    return tf.layers.dense(inputs, 1000)

In [19]:
input_train_fn = lambda:input_fn(is_training=True, data_dir=DATA_DIR, 
                          batch_size=per_device_batch_size(BATCH_SIZE, 1),
                          num_epochs=NUM_EPOCHS,
                          num_gpus=1)

In [20]:
model_fn = lambda features, labels, mode :run_model_fn(features, labels, mode, 
                                                          alexnet_model, "classic_alexnet_bn", True, 
                                                          False, "channels_first")

In [21]:
# distribution = tf.contrib.distribute.OneDeviceStrategy('device:GPU:0')
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
run_config = tf.estimator.RunConfig(session_config=config)
classifier = tf.estimator.Estimator(model_fn=model_fn, 
                                    model_dir="./tmp/classic_alexnet_bn",
                                    config=run_config,
                                    params=None)

INFO:tensorflow:Using config: {'_model_dir': './tmp/classic_alexnet_bn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  allow_growth: true
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4a34449470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
classifier.train(input_fn=input_train_fn, hooks=[],
                 max_steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


ResourceExhaustedError: OOM when allocating tensor of shape [4096,4096] and type float
	 [[Node: dense_1/kernel/Momentum/Initializer/zeros = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [4096,4096] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'dense_1/kernel/Momentum/Initializer/zeros', defined at:
  File "/home/karen/anaconda3/envs/develop/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/karen/anaconda3/envs/develop/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/karen/anaconda3/envs/develop/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/karen/anaconda3/envs/develop/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/karen/anaconda3/envs/develop/lib/python3.6/site-packages/ipykernel/kernelapp.py", line

### Innovations

1. first use of ReLU
2. Used LRNorm Layers
3. Heavy Data Augumentation
4. Dropout 0.5
5. batchSize 128
6. SGD Momentum 0.9
7. Learning Rate:  1e-2, reduced by 10 manually when val accuracy plateaus
8. L2 weight decay 5e-4
9. 7 CNN Ensemble